## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict

pd.options.mode.chained_assignment = None

In [2]:
os.listdir()

['.DS_Store', 'MIMIC', 'mimic3.ipynb', '.ipynb_checkpoints']

In [3]:
os.chdir('./MIMIC')
paths = sorted(os.listdir())
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('PROCEDURES_ICD.csv')
paths.remove('PROCEDUREEVENTS_MV.csv')
paths.remove('MICROBIOLOGYEVENTS.csv')
paths.remove('D_CPT.csv')
paths

['ADMISSIONS.csv',
 'CHARTEVENTS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_CV_Filtered1.csv',
 'INPUTEVENTS_CV_Filtered2.csv',
 'INPUTEVENTS_CV_Filtered3.csv',
 'INPUTEVENTS_CV_FilteredPDA.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'LABEVENTS_Filtered.csv',
 'LABEVENTS_FilteredPDA.csv',
 'OUTPUTEVENTS.csv',
 'OUTPUTEVENTS_Filtered.csv',
 'OUTPUTEVENTS_FilteredPDA.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv']

## Load Data

In [4]:
datas = []
for path in paths:
    datas.append(pd.read_csv(path))

/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1517/1416101455.py:3: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(path))
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1517/1416101455.py:3: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(path))
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1517/1416101455.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(path))
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1517/1416101455.py:3: DtypeWarning: Columns (9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(path))
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1517/1416101455.py:3: DtypeWarning: Columns (9,17,2

## Clean Data to remain SEPSIS patient and sort Data

In [5]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv(paths[i])
D_ICD_DIAGNOSES

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
10304,11403,99591,Sepsis,Sepsis
10305,11404,99592,Severe sepsis,Severe sepsis
13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [ ]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
'''
drop_index = []
for j in DIAGNOSES_ICD.index:
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv(paths[i])
'''
DIAGNOSES_ICD

In [24]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


In [ ]:
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in range(len(datas)):
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            for j in range(len(datas[i][column])):
                datas[i][column].iloc[j] = pd.Timestamp(datas[i][column].iloc[j])
    datas[i].to_csv(paths[i])
del dict1

        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv(paths[i])

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(paths[i])

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(paths[i])

/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1230/2303490204.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas[i][column].iloc[j] = pd.Timestamp(datas[i][column].iloc[j])
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1230/2303490204.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas[i][column].iloc[j] = pd.Timestamp(datas[i][column].iloc[j])
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1230/2303490204.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

# Look Data characteristics

In [8]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [9]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


In [15]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [10]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17422687,27946,169481.0,212466.0,2154-07-16 22:00:00,40864,82.0,ml,NaN,NaN,...,10296124,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527931,17422688,27946,169481.0,212466.0,2154-07-16 23:00:00,40864,82.0,ml,NaN,NaN,...,10248887,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527932,17422689,27946,169481.0,212466.0,2154-07-17 00:00:00,40864,NaN,ml,NaN,NaN,...,7418343,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527933,17422690,30516,132490.0,260091.0,2105-08-25 06:30:00,46046,31.0,ml,NaN,NaN,...,1753414,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
INPUTEVENTS_CV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [11]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_MV = datas[i].copy()
INPUTEVENTS_MV

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17422687,27946,169481.0,212466.0,2154-07-16 22:00:00,40864,82.0,ml,NaN,NaN,...,10296124,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527931,17422688,27946,169481.0,212466.0,2154-07-16 23:00:00,40864,82.0,ml,NaN,NaN,...,10248887,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527932,17422689,27946,169481.0,212466.0,2154-07-17 00:00:00,40864,NaN,ml,NaN,NaN,...,7418343,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527933,17422690,30516,132490.0,260091.0,2105-08-25 06:30:00,46046,31.0,ml,NaN,NaN,...,1753414,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
INPUTEVENTS_MV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [12]:
i = paths.index('OUTPUTEVENTS.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,4340476,68375,123645.0,276251.0,2175-09-05 23:00:00,226588,30.0,mL,2175-09-05 22:48:00,16915,NaN,NaN,NaN
4349214,4340477,68375,123645.0,276251.0,2175-09-06 00:00:00,226588,0.0,mL,2175-09-06 00:03:00,20971,NaN,NaN,NaN
4349215,4340478,68375,123645.0,276251.0,2175-09-06 01:00:00,226588,40.0,mL,2175-09-06 01:13:00,20971,NaN,NaN,NaN
4349216,4340479,68375,123645.0,276251.0,2175-09-06 02:00:00,226588,20.0,mL,2175-09-06 02:14:00,20971,NaN,NaN,NaN


In [54]:
i = paths.index('D_ITEMS.csv')
D_ITEMS = datas[i].copy()
D_ITEMS

,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,290,27,1,% Inspir. Time,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,291,28,2,ABI (L),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,292,29,3,ABI (R),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,293,30,4,ABI Ankle BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,294,31,5,ABI Brachial BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12482,9505,15456,228643,Barrier precautions in place (Intubation),Barrier precautions in place (Intubation),metavision,chartevents,Intubation,NaN,Text,NaN
12483,9506,15457,228644,Barrier precautions in place (LP),Barrier precautions in place (LP),metavision,chartevents,Lumbar Puncture,NaN,Text,NaN
12484,9507,15458,228645,Barrier precautions in place (PACEN),Barrier precautions in place (PACEN),metavision,chartevents,Paracentesis,NaN,Text,NaN
12485,9508,15459,228646,Barrier precautions in place (THCEN),Barrier precautions in place (THCEN),metavision,chartevents,Thoracentesis,NaN,Text,NaN


In [55]:
for i in range(len(D_ITEMS)):
    if D_ITEMS.iloc[i]['ITEMID'] == 30025:
        print(D_ITEMS.iloc[i])

Unnamed: 0                5782
ROW_ID                    5352
ITEMID                   30025
LABEL                  Heparin
ABBREVIATION               NaN
DBSOURCE               carevue
LINKSTO         inputevents_cv
CATEGORY                   NaN
UNITNAME                   NaN
PARAM_TYPE                 NaN
CONCEPTID                  NaN
Name: 5056, dtype: object


In [56]:
i = paths.index('D_LABITEMS.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS

,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,127,1,50800,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,129,3,50802,Base Excess,Blood,Blood Gas,11555-0
3,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
4,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6
...,...,...,...,...,...,...,...
748,748,749,51551,VOIDED SPECIMEN,OTHER BODY FLUID,HEMATOLOGY,NaN
749,749,750,51552,VOIDED SPECIMEN,STOOL,CHEMISTRY,NaN
750,750,751,51553,VOIDED SPECIMEN,URINE,CHEMISTRY,NaN
751,751,752,51554,VOIDED SPECIMEN,JOINT FLUID,HEMATOLOGY,NaN


In [59]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,26625,22326,21,109451,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
1,26624,22325,21,109451,APR,1744,Percutaneous Cardiovascular Procedures w/ AMI,4.0,4.0
2,26584,25700,21,111970,APR,7204,Septicemia & Disseminated Infections,4.0,4.0
3,26583,25699,21,111970,HCFA,416,SEPTICEMIA AGE >17,NaN,NaN
4,15712,11708,33,176176,HCFA,416,SEPTICEMIA AGE >17,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7659,115484,115927,99768,140641,APR,2794,Hepatic Coma & Other Major Acute Liver Disorders,4.0,4.0
7661,115486,115929,99768,140641,MS,441,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
7663,63568,63407,99973,150202,APR,2483,Major Gastrointestinal & Peritoneal Infections,3.0,3.0
7662,63566,63405,99973,150202,APR,2483,Major Gastrointestinal & Peritoneal Infections,3.0,2.0


In [13]:
i = paths.index('LABEVENTS.csv')
LABEVENTS = datas[i].copy()
LABEVENTS

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [14]:
i = paths.index('PRESCRIPTIONS.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,5.601728e+07,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,5.482972e+07,20mg Tablet,20,mg,1,TAB,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156445,3996662,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED20,006751,5.400182e+07,20 mg Tablet,40,mg,2,TAB,PO/NG
4156446,3996070,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5mL Vial,1,NEB,1,VIAL,IH
4156447,3996063,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone,HYDR20/100NS,048078,6.155302e+10,20 mg / 100 mL Premix Bag,0.12,mg,0.01,BAG,IVPCA
4156448,3996062,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,Docusate Sodium,Docusate Sodium,Docusate Sodium,DOCU100,003009,9.042245e+08,100mg Capsule,100,mg,1,CAP,PO


# State

In [21]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Time

In [123]:
data = ADMISSIONS[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME']]
ADMISSIONS.insert(2, 'ICUSTAY_ID', pd.Series(['NaN'] * len(ADMISSIONS)))
for i in range(len(ADMISSIONS)):
    subject_id = data.iloc[i]['SUBJECT_ID']
    hadm_id = data.iloc[i]['HADM_ID']
    admittime = data.iloc[i]['ADMITTIME']
    dischtime = data.iloc[i]['DISCHTIME']
    icustays = ICUSTAYS[(ICUSTAYS['SUBJECT_ID'] == subject_id) & (ICUSTAYS['HADM_ID'] == hadm_id)]
    if len(icustays) > 1:
        print(subject_id)
    ADMISSIONS['ICUSTAY_ID'][i] = icustays['ICUSTAY_ID'].values[0]

590
914
940
1193
1329
1380
1579
1621
1719
1795
1999
2053
2397
2743
3294
3866
3866
3941
4109
4916
5070
5362
5369
5495
6131
7328
7478
7478
7666
7853
8060
8086
9002
10399
10624
10653
13207
14008
14112
14191
14757
15066
15219
15243
15454
16351
17006
17355
17483
17722
18033
18740
19424
19692
19729
20354
20592
20941
21948
21990
22187
22648
22949
23038
23680
24142
24359
24623
24846
25111
25312
25498
27232
27427
27983
28165
28166
28315
28477
28702
28926
29035
29276
29968
30798
40308
41517
41547
42757
44500
45310
46588
47724
48042
48292
48351
49544
54247
55328
55971
60659
60897
64334
66474
67735
73713
74857
78531
79651
81491
81660
83607
83751
86276
88308
89292
89927
90143
90479
94767
96240
98220


In [126]:
ICUSTAYS[ICUSTAYS['SUBJECT_ID'] == 590]

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
62,1141,758,590,172260,235703,carevue,MICU,MICU,52,52,2124-11-02 08:58:00,2124-11-04 21:33:00,2.5243
63,1142,759,590,172260,258414,carevue,MICU,MICU,52,52,2124-11-05 22:40:00,2124-11-15 18:29:00,9.8257


In [127]:
ADMISSIONS[ADMISSIONS['SUBJECT_ID'] == 590]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ADMITTIME,DISCHTIME,DEATHTIME
33,590,172260,235703,2124-11-02 08:56:00,2124-11-15 18:29:00,2124-11-15 18:29:00


In [182]:
a = ICUSTAYS[ICUSTAYS['SUBJECT_ID'] == 590]['INTIME']
pd.Timestamp(ICUSTAYS['INTIME'])

TypeError: Cannot convert input [0       2134-09-11 20:50:04
1       2135-01-30 20:53:34
2       2116-12-23 22:31:53
3       2166-08-10 00:29:36
4       2113-02-15 00:20:44
               ...         
3727    2191-08-03 19:12:37
3726    2191-12-23 01:05:09
3728    2191-06-15 17:54:36
3729    2122-02-21 01:41:25
3730    2180-11-27 02:30:43
Name: INTIME, Length: 3701, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp

In [172]:
from datetime import datetime
ts = pd.Timestamp(ICUSTAYS[ICUSTAYS['SUBJECT_ID'] == 590]['INTIME'][62])
datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

TypeError: an integer is required (got type Timestamp)

In [173]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

### shock_index

In [31]:
for s in D_LABITEMS['LABEL']:
    if 'SHOCK_INDEX' in str(s):
        print(s)

### exlixhauser

### SIRS

In [35]:
for s in D_LABITEMS['LABEL']:
    if 'SIRS' in str(s):
        print(s)